In [138]:
rm(list=ls())

# Code should be ready to work with:

### MATCHING DATA
id_individuals <- rep(1:6,each=20)
n_individuals <- length(unique(id_individuals))
id_sessions <- rep(1:20,6)
n_obs <- length(id_individuals)
n_sessions <- length(unique(id_sessions))# Assume same number of sessions acroos birds?? What about stable??
## Count data
Bl <- rpois(n=n_individuals*n_sessions,60)# Behavior (response counts) left
Br <- rpois(n=n_individuals*n_sessions,60)# Behavior right
Wl <- rpois(n=n_individuals*n_sessions,10)# Rewards left
Wr <- rpois(n=n_individuals*n_sessions,10)# Rewards right
observed_counts <- c('id_individuals','id_sessions',
                     'n_obs','n_sessions','n_individuals',
                       'Bl','Br','Wl','Wr')
## Binary data
#chosen_option
#rewarded

In [139]:
head(data.frame(id_individuals,id_sessions,Bl,Br,Wl,Wr),3)

,id_individuals,id_sessions,Bl,Br,Wl,Wr
,<int>,<int>,<int>,<int>,<int>,<int>
1,1,1,58,66,16,12
2,1,2,57,53,11,13
3,1,3,58,62,15,13


In [140]:
### PREDICTOR DATA
X <- array(dim=c(n_obs,3)) # Desing matrix n_obs x n_pred; mixing obs from all ids, long format
X[,1] <- 1; X[,2] <- rnorm(n_obs,0,1); X[,3] <- rnorm(n_obs,2,1) 
n_pred <- ncol(X)
zeros_pred <- rep(0,n_pred) # Prior coefficients
I_pred <- diag(1,ncol=n_pred,nrow=n_pred) # Prior coefficients
zeros_errors <- rep(0,n_sessions) # Prior errors
I_errors <- diag(1,ncol=n_sessions,nrow=n_sessions) # Prior errors  
observed_explanatory <- c('X','n_pred',
                             'zeros_pred','I_pred',
                             'zeros_errors','I_errors')

In [141]:
length(zeros_pred)
dim(I_pred)
length(zeros_errors)
dim(I_errors)

[1] 3

[1] 3 3

[1] 20

[1] 20 20

In [142]:
explanatory <- ' # model start
### Explanatory model
xi_a ~ dmnorm(zeros_pred,0.01*I_pred)
xi_b ~ dmnorm(zeros_pred,0.01*I_pred)
#sigma_e_a ~ dunif(0,50)
#sigma_e_b ~ dunif(0,50)
for(b in 1:n_individuals){
    epsilon_a[b,1:n_sessions] ~ dmnorm(zeros_errors,
                                       I_errors) # Explanatory; unobserved sessions not updated?
    epsilon_b[b,1:n_sessions] ~ dmnorm(zeros_errors,
                                       I_errors) # Explanatory
}
for(i in 1:n_obs){
#    alpha[birds[i],sessions[i]] <- xi0_a + xi1_a*weight[i] + epsilon_a[birds[i],sessions[i]]
#    beta[birds[i],sessions[i]] <- xi0_b + xi1_b*weight[i] + epsilon_b[birds[i],sessions[i]]
    eta_a[i] <- X[i,]%*%xi_a
    eta_b[i] <- X[i,]%*%xi_b
    alpha[id_individuals[i],id_sessions[i]] <- eta_a[i] + epsilon_a[id_individuals[i],id_sessions[i]]
    beta[id_individuals[i],id_sessions[i]] <- eta_b[i] + epsilon_b[id_individuals[i],id_sessions[i]]
}
# model end'
 
matching <- ' # model start
### Core matching model
shape_tau_prior <- 2
rate_tau_prior <- 1
for(b in 1:n_individuals){
    tau[b]   ~ dgamma(shape_tau_prior, rate_tau_prior)T(0.01,) # Matching COUNT model
}

for(i in 1:n_obs){
    lambda_Br[i] ~ dlnorm( alpha[id_individuals[i],id_sessions[i]]/2 
                          + beta[id_individuals[i],id_sessions[i]] 
                          * log(Wr[i])/2, 
                          tau[id_individuals[i]])
    lambda_Bl[i] ~ dlnorm(-alpha[id_individuals[i],id_sessions[i]]/2 
                          - beta[id_individuals[i],id_sessions[i]] 
                          * log(Wl[i])/2, 
                          tau[id_individuals[i]])
    Br[i] ~ dpois(lambda_Br[i])
    Bl[i] ~ dpois(lambda_Bl[i])
    # Posterior predictive
    Br_post[i] ~ dpois(lambda_Br[i])
    Bl_post[i] ~ dpois(lambda_Bl[i])
}
# model end'

write(paste('model{',explanatory,matching,'}',sep='\n'),'factory_matching.bug')

In [143]:
unobserved_explanatory <- c('xi_a','xi_b',
                          #  'sigma_e_a','sigma_e_b',
                            'eta_a','eta_b',
                            'epsilon_a','epsilon_b')
unobserved_counts <- c('alpha', 'beta', 'tau',
                       'lambda_Br', 'lambda_Bl','Br_post','Bl_post')

observed <- list(c(observed_counts,observed_explanatory))[[1]]
observed <- list(c(observed_counts,observed_explanatory))[[1]]
unobserved <- c(unobserved_explanatory,unobserved_counts)

In [144]:
library('R2jags')
bayes <- jags(data = observed,
             parameters.to.save = unobserved,
             model.file = 'factory_matching.bug')

Warning message in jags.model(model.file, data = data, inits = init.values, n.chains = n.chains, :
“Unused variable "n_pred" in data”


Compiling model graph
   Resolving undeclared variables
   Allocating nodes
Graph information:
   Observed stochastic nodes: 240
   Unobserved stochastic nodes: 500
   Total graph size: 3939

Initializing model

